# Data
Download the IMDB reviews if its not already in the local cache.
Load from the local cache and split to train and test sets.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import os
import ssl
from tensorflow.keras.utils import get_file
try:
    from notebooks.data import load_imdb
except ModuleNotFoundError:
    from data import load_imdb

if (not os.environ.get('PYTHONHTTPSVERIFY', '')
        and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

zip_file = get_file('aclImdb.zip', origin='http://mng.bz/0tIo', extract=True)
imdb_dir = zip_file.replace('.zip', '')
(train_texts, train_labels), (test_texts, test_labels) = load_imdb(imdb_dir)

print('An example from the reviews:')
print('============================')
print(f'Text : {train_texts[0][:200]}...')
print(f'Label: {train_labels[0]}')

An example from the reviews:
Text : Entertainment Tonight has been going down hill for the last few years, but as of last night (Aug 17th 2006) they reached a new low.<br /><br />In an effort to try to hype up their broadcast, they deci...
Label: neg


# Prepare Data
Initialize a `WordVectorizer`. The vectorizer will be fitted on the 
`train_texts`. We won't use any word tokenizer, so the words will be splitted
on spaces and keep the `num_words` most common words.

In [3]:
from keras_nlp import WordVectorizer

num_words = 50000
word_vectorizer = WordVectorizer(num_words=num_words, oov_token='UNK')
word_vectorizer.fit_on_texts(train_texts)
# The vectorizer's number of tokens num_words + PAD + OOV token
print(f'Vectorizer number of tokens: {word_vectorizer.num_tokens}')

2020-Apr-10 00:06:11 [INFO    :WordVectorizer] - Creating vocabulary.


25000/25000 [==============================] - 4s 144us/step
Vectorizer number of tokens: 50002


## Encode Labels
The labels are strings ('pos' / 'neg'). We will convert them to numbers.

In [4]:
from sklearn.preprocessing import label_binarize

y_train = label_binarize(train_labels, classes=['neg', 'pos'])
y_test = label_binarize(test_labels, classes=['neg', 'pos'])

## Keep a Validation Set

In [5]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, y_train, y_val = train_test_split(
    train_texts, y_train, stratify=train_labels, random_state=45)

## Create Generators for Vectorized Data
We use the `texts_to_vectors_generator` method. The generator expects the 
texts, the labels and other parameters such as `shape` and `batch_size`.
For the `shape` we define to be `max_tokens` per sentences.

**Attention**: We *must* pass the same number of `max_tokens` when 
converting different text sets (train, validation, test). 
If we don't, then it is almost certain that the results will have different 
numbers of columns because the set are likely to have different number of tokens.
We create *generators* for the train, validation and test text sets.


In [6]:
max_tokens = 500
batch_size = 32
train_generator = word_vectorizer.texts_to_vectors_generator(
    train_texts, y_train, shape=(max_tokens,), batch_size=batch_size)
val_generator = word_vectorizer.texts_to_vectors_generator(
    val_texts, y_val, shape=(max_tokens,), batch_size=batch_size)

# Neural Network
Here we define a toy network for demonstration purpose.

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense

words_input = Input(shape=(max_tokens, ), name='Input', dtype='int32')
words_embeddings = Embedding(
    input_dim=len(word_vectorizer.token2id),
    output_dim=100,
    input_length=max_tokens,
    mask_zero=0,
    trainable=True,
    name='Embeddings')(words_input)
x = Flatten(name='Flatten')(words_embeddings)
predictions = Dense(1, activation='sigmoid', name='Predictions')(x)
model = Model(words_input, predictions)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 500)]             0         
_________________________________________________________________
Embeddings (Embedding)       (None, 500, 100)          5000200   
_________________________________________________________________
Flatten (Flatten)            (None, 50000)             0         
_________________________________________________________________
Predictions (Dense)          (None, 1)                 50001     
Total params: 5,050,201
Trainable params: 5,050,201
Non-trainable params: 0
_________________________________________________________________
None


## Train Model

In [ ]:
steps_per_epoch = len(train_texts) // batch_size
if steps_per_epoch % batch_size:
    steps_per_epoch += 1
val_steps_per_epoch = len(val_texts) // batch_size
if val_steps_per_epoch % batch_size:
    val_steps_per_epoch += 1

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=val_generator,
                    validation_steps=val_steps_per_epoch,
                    epochs=5)

  ...
    to  
  ['...']


2020-Apr-10 00:06:15 [WARNING :tensorflow  ] - sample_weight modes were coerced from
  ...
    to  
  ['...']


  ...
    to  
  ['...']


2020-Apr-10 00:06:15 [WARNING :tensorflow  ] - sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 586 steps, validate for 196 steps
Epoch 1/5
586/586 [==============================] - 38s 64ms/step - loss: 0.4230 - acc: 0.7963 - val_loss: 0.3272 - val_acc: 0.8600
Epoch 2/5
586/586 [==============================] - 40s 69ms/step - loss: 0.1140 - acc: 0.9644 - val_loss: 0.2685 - val_acc: 0.8923
Epoch 3/5
586/586 [==============================] - 38s 65ms/step - loss: 0.0271 - acc: 0.9959 - val_loss: 0.2966 - val_acc: 0.8842
Epoch 4/5
538/586 [==========================>...] - ETA: 2s - loss: 0.0079 - acc: 0.9997

## Model Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

steps = len(test_texts) // batch_size
if len(test_texts) % batch_size:
    steps += 1
test_generator = word_vectorizer.texts_to_vectors_generator(
    test_texts, y_test, shape=(max_tokens,), batch_size=batch_size)
predictions = model.predict(test_generator, steps=steps, verbose=1)
y_pred = np.round(predictions)
print(classification_report(
    y_test, y_pred, digits=4, target_names=['neg', 'pos']))

# Plot Accuracy and Loss

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

_, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(12, 6))
ax1.plot(history.epoch, history.history['acc'], label='Training')
ax1.plot(history.epoch, history.history['val_acc'], label='Validation')
ax1.set_title('Accuracy')
ax1.legend()

ax2.plot(history.epoch, history.history['loss'], label='Training')
ax2.plot(history.epoch, history.history['val_loss'], label='Validation')
ax2.set_title('Loss')
ax2.set_xlabel('epochs')
ax2.legend()

plt.show()